In [1]:
import pandas as pd

df = pd.read_csv('allergen_synonyms.csv')
df.sort_values(by=['Name'], ascending=True, inplace=True)

#print(df.head())
#print(df.tail())
#print(df.shape)

print(list(df.Name))

['Alpha-Tocopherol', 'Alpha-Tocopherol-Acetate', 'Amidoamine', 'Bacitracin', 'Balsam of Peru', 'Balsam of Peru', 'Benzalkonium Chloride', 'Benzocaine', 'Black Rubber Mix', 'Bromo-2-Nitropropaine-1,3 Diol', 'Budesonide', 'Carba Mix', 'Chloroxylenol (PCMX)', 'Cinnamic Aldehyde', 'Cobalt Dichloride', 'Cocamindopropyl Betaine', 'Coconut Diethanolamide', 'Colophony', 'DMDM Hydantoin', 'Diazolidinyl Urea', 'Dibucaine hydrochloride', 'Dimethylol Dihydroxyethylene Urea', 'Epoxy Resin', 'Ethyl Acrylate', 'Ethylene Urea', 'Ethylenediamine Dihydrochloride (EDD)', 'Euxyl K400', 'Formaldehyde', 'Formaldehyde', 'Fragrance Mix', 'Glutaraldehyde', 'Glyceryl Thioglycolate', 'Gold Sodium Thiosulfate', 'Hydrocortisone-17-Butyrate', 'Hydroxy-4-Methoxybenzophenone', 'Imidazolidnyl Urea', 'Iodopropyl Butylcarbamate', 'Jasmine Absolute', 'Lanolin Alcohol', 'Lidocaine', 'Mercapto Mix', 'Mercaptobenzothiazole', 'Methyl Methacrylate', 'Methylchloroisothiazolinone', 'Methyldibromoglutaronitrile / Phenoxyethanol'

In [3]:
df.head(1)


,Name,Synonyms
0,Alpha-Tocopherol,"Alpha-tocopherol, d-alpha-tocopheryl acetate, ..."
